In [70]:
# Import Library
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
from pymongo import MongoClient

from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

In [71]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI=os.getenv("MONGODB_URI")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

In [72]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
)

In [73]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq']

In [74]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
)

In [75]:
# Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Resmi Allo Bank Indonesia.
    Tugas Anda adalah menjawab pertanyaan berdasarkan data FAQ resmi dari website Allo Bank.

    Tanggung jawab utama:
    Memberikan jawaban akurat dan informatif sesuai dengan kategori FAQ: umum, akun, produk, layanan, proses pengaduan nasabah, dan registrasi.
    Menyampaikan informasi dengan bahasa profesional, jelas, dan mudah dipahami.
    Memberikan panduan teknis (jika diperlukan) dalam format numerik atau langkah-langkah berurutan.
    Menanggapi keluhan atau kendala nasabah dengan solusi yang relevan dari FAQ.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan bahwa Anda tidak memiliki informasi tersebut.
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal, tautan promosi, atau informasi tambahan di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi:
    {context}

    Pertanyaan: {question}

    Jawaban profesional:
    """
)

In [ ]:
# Model Configuration dengan GPT-3.5 turbo
llm_gpt35 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=OPENAI_KEY,
    temperature=0,
    max_tokens=500
)

In [ ]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_gpt35,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.78
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
)

In [78]:
faq_df = pd.read_csv('/Users/ilham/OneDrive/Desktop/finalprojectrmt41/data/faq_allobank_clean.csv')

def answer_from_doc(question):
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

# <b>Pertanyaan di dalam konteks FAQ Allo Bank secara Formal dan sesuai Database</b>

In [79]:
# Test Cases 1
questions = [
    "Apa itu Allo Bank?",
    "Mengapa saya tidak bisa menggunakan Allo Pay saya?",
    "Mengapa saya tidak bisa log in di aplikasi Allo Bank?",
    "Apakah menyimpan dana di Allo Bank dipastikan aman?",
    "Mengapa aplikasi Allo Bank saya berjalan sangat lambat?",
    "Bagaimana jika saldo atau limit saya tidak cukup?",
    "Keuntungan apa yang saya dapatkan sebagai member Allo Bank?",
    "Mengapa saya tidak bisa mendaftarkan nomor ponsel/email di aplikasi Allo Bank?",
    "Biaya apa saja yang dikenakan ketika melakukan registrasi?",
    "Mengapa saya harus mengisi informasi pribadi saat proses registrasi?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Allo Bank adalah bank yang didirikan di Indonesia pada tahun 2021. Allo Bank terbentuk ketika Bank Harda Internasional (BHI) diakuisisi oleh Mega Corpora, dan setelah diakuisisi, Bank Harda Internasional (BHI) secara resmi berganti nama menjadi PT. Allo Bank.

ACTUAL DATA: Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan "All in One" dan mempunyai arti sebagai "one for all, and all for one" alias semua untuk satu dan satu untuk semua.

Pertanyaan: Mengapa saya tidak bisa menggunakan Allo Pay saya?
Untuk dapat menggunakan Allo Pay, pastikan Anda telah melakukan proses registrasi Allo Pay dan telah login pada aplikasi Allo Bank. Jika Anda masih mengalami kendala, silakan ikuti langkah-langkah berikut:
1. Buka aplikasi Allo Bank Anda.
2. Tekan tombol "Perbarui Sekarang" pada Ber

Berdasarkan uji coba menggunakan model GPT-3.5-turbo dengan pertanyaan FAQ resmi yang disampaikan secara formal dalam konteks Allo Bank, dan di mana pertanyaan tersebut memang tersedia dalam database, diketahui bahwa model mampu memberikan jawaban yang relevan dan tepat sesuai dengan informasi yang tersimpan di database.

# <b>Pertanyaan di dalam konteks FAQ Allo Bank secara Informal dan tidak sesuai Database</b>

In [80]:
# Test Cases 2
questions = [
    "Allo Bank itu apa sih?",
    "Kenapa ya aku nggak bisa pakai Allo Pay?",
    "Aku nggak bisa login ke aplikasi Allo Bank, kenapa ya?",
    "Nyimpen uang di Allo Bank aman nggak sih?",
    "Kenapa aplikasi Allo Bank aku lemot banget?",
    "Kalau saldo atau limit aku nggak cukup, gimana ya?",
    "Kalau jadi member Allo Bank, aku dapet keuntungan apa aja sih?",
    "Kenapa aku nggak bisa daftar pakai nomor HP atau email di aplikasi Allo Bank?",
    "Pas daftar, ada biaya yang harus dibayar nggak sih?",
    "Kenapa sih aku harus isi data pribadi waktu daftar?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Allo Bank itu apa sih?
Allo Bank adalah bank yang didirikan di Indonesia pada tahun 2021. Bank ini terbentuk setelah Bank Harda Internasional (BHI) diakuisisi oleh Mega Corpora dan berganti nama menjadi PT. Allo Bank. Allo Bank menawarkan berbagai produk dan layanan keuangan, termasuk produk tabungan digital seperti Allo Prime yang dapat diakses melalui aplikasi Allo Bank atau aplikasi Business Unit yang berada di bawah naungan CT Corp.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Kenapa ya aku nggak bisa pakai Allo Pay?
Untuk menggunakan fitur Allo PayLater, pastikan Anda telah melakukan langkah-langkah berikut:
1. Login pada aplikasi Allo Bank.
2. Tekan tombol "Allo PayLater" pada halaman beranda aplikasi Allo Bank.

Jika setelah melakukan langkah-langkah tersebut Anda masih mengalami kendala dalam menggunakan Allo PayLater, Anda dapat menghubungi layanan pelanggan Allo Bank untuk bantuan lebih lanjut. Terima kasih.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Aku ngga

Dalam pengujian chatbot menggunakan model GPT-3.5-turbo, ditemukan bahwa meskipun pertanyaan yang diajukan bersifat informal dan tidak persis sama dengan pertanyaan yang ada di database FAQ, model tetap mampu memberikan jawaban yang relevan dan akurat. Jawaban yang dihasilkan oleh model konsisten dengan informasi yang terdapat di database FAQ resmi.

# <b>Pertanyaan di luar Konteks FAQ Allo Bank namun masih berkaitan dengan banking</b>

In [84]:
# Test Cases 3
questions = [
    "Bagaimana cara kerja sistem kredit di bank secara umum?",
    "Apa perbedaan antara rekening tabungan dan rekening giro?",
    "Bagaimana prosedur pengajuan KPR di bank lain selain Allo Bank?",
    "Apa itu suku bunga floating dan fixed dalam pinjaman?",
    "Bagaimana keamanan transaksi perbankan melalui aplikasi mobile banking?",
    "Apa yang dimaksud dengan layanan internet banking?",
    "Bagaimana cara mengajukan kartu kredit baru di bank konvensional?",
    "Apa saja biaya yang biasanya dikenakan pada rekening koran?",
    "Bagaimana cara cek mutasi rekening melalui ATM?",
    "Apa itu BI Checking dan bagaimana pengaruhnya terhadap pengajuan kredit?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Bagaimana cara kerja sistem kredit di bank secara umum?
Sistem kredit di bank umumnya bekerja dengan memberikan pinjaman kepada nasabah berdasarkan sejumlah faktor, seperti riwayat kredit, pendapatan, dan kemampuan membayar. Prosesnya melibatkan penilaian risiko oleh bank untuk menentukan apakah nasabah layak mendapatkan pinjaman. Setelah pinjaman disetujui, nasabah akan membayar kembali pinjaman beserta bunga sesuai dengan kesepakatan yang telah ditetapkan. Untuk menjaga kredibilitas pinjaman, penting untuk melakukan pembayaran tepat waktu sesuai dengan jadwal yang telah ditentukan.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Apa perbedaan antara rekening tabungan dan rekening giro?
Rekening tabungan adalah jenis rekening yang biasanya digunakan untuk menyimpan dana dengan tujuan untuk menabung. Dana yang disimpan dalam rekening tabungan cenderung tidak sering digunakan untuk transaksi sehari-hari. Sementara itu, rekening giro adalah jenis rekening yang biasanya digunak

Berdasarkan pengujian model GPT-3.5-turbo terhadap pertanyaan di luar Konteks FAQ Allo Bank namun masih berkaitan dengan banking diketahui bahwa model mampu untuk berusaha menjawab beberapa pertanyaan umum banking dengan infromasi yang berlandaskan dari website Allo Bank, namun juga terdapat beberapa pertanyaan di luar konteks Allo Bank yang dijawab oleh model dengan informasi yang tidak relevan sesuai dengan batasan konteks project seperti BI Checking, terdapat juga pertanyaan yang ditolak oleh model yang informasinya tidak relevan dengan batasan konteks project seperti mengajukan kartu kredit baru di bank konvensional dan prosedur pengajuan KPR di bank lain selain Allo Bank.

# <b>Pertanyaan di luar Konteks Allo Bank</b>

In [83]:
# Test Cases 4
questions = [
    "Siapa presiden dan wakil presiden pertama Indonesia?",
    "Kapan Indonesia merdeka?",
    "Apakah alien itu benar-benar ada?",
    "Bagaimana cara belajar coding untuk pemula?",
    "Siapa penemu bom atom?",
    "Siapa itu Oppenheimer?",
    "Siapa itu Christoper Nolan?",
    "Apa itu teori relativitas?",
    "Apakah black hole itu nyata?",
    "Mengapa bumi itu bulat?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(result['result'])
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Siapa presiden dan wakil presiden pertama Indonesia?
Maaf, pertanyaan tersebut tidak relevan dengan informasi yang tersedia dalam FAQ resmi Allo Bank Indonesia. Jika Anda memiliki pertanyaan lain terkait dengan produk, layanan, atau prosedur di Allo Bank, silakan sampaikan kepada saya. Terima kasih.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Kapan Indonesia merdeka?
Mohon maaf, pertanyaan tersebut tidak relevan dengan informasi yang tersedia dalam FAQ resmi Allo Bank Indonesia. Jika Anda memiliki pertanyaan lain terkait layanan atau produk kami, silakan berikan informasi lebih detail agar kami dapat memberikan jawaban yang sesuai. Terima kasih.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Apakah alien itu benar-benar ada?
Maaf, pertanyaan tersebut tidak relevan dengan informasi yang tersedia dalam FAQ resmi Allo Bank Indonesia. Saya tidak memiliki informasi terkait hal tersebut. Jika Anda memiliki pertanyaan lain terkait produk atau layanan Allo Bank, silakan berikan p

Berdasarkan pengujian model GPT-3.5-turbo terhadap pertanyaan di luar konteks Allo Bank, diketahui bahwa model mampu menolak dengan cara yang sopan, ramah, dan profesional terhadap pertanyaan yang tidak relevan dan tidak terdapat dalam database FAQ. Selain itu, model juga memberikan informasi mengenai batasan konteks prompting yang digunakan.

# <b>Kesimpulan</b>

Berdasarkan serangkaian pengujian terhadap chatbot yang dibangun menggunakan model <b>GPT-3.5-turbo</b> dan berbasis pada database FAQ resmi Allo Bank, dapat disimpulkan bahwa:

1. <b>Kemampuan Memahami Variasi Pertanyaan</b>,
Chatbot mampu memberikan jawaban yang relevan dan akurat terhadap pertanyaan resmi maupun informal yang masih sesuai dengan konteks FAQ Allo Bank. Hal ini menunjukkan bahwa model tidak hanya melakukan pencocokan teks secara langsung, tetapi juga mampu memahami konteks dan makna dari pertanyaan yang disampaikan.

2. <b>Kemampuan Menjaga Batasan Konteks</b>,
Dalam beberapa kasus pertanyaan di luar cakupan FAQ Allo Bank namun masih berkaitan dengan topik perbankan umum, model berusaha menjawab dengan mengacu pada informasi dari website Allo Bank. Meskipun demikian, terdapat pula kasus di mana model memberikan jawaban yang tidak relevan dengan batasan konteks proyek, seperti pertanyaan mengenai BI Checking atau layanan dari bank lain.

3. <b>Kemampuan Menolak dengan Tepat</b>,
Model menunjukkan kemampuan untuk menolak menjawab pertanyaan yang berada di luar konteks database FAQ Allo Bank dengan cara yang sopan, ramah, dan profesional. Ini sesuai dengan tujuan proyek dalam membatasi cakupan informasi hanya pada sumber resmi yang tersedia.

<b>Model GPT-3.5-turbo</b>  

Kelebihan :  
- Dapat menjawab pertanyaan dengan relevan dan akurat.
- Dapat memahami pertanyaan baik secara formal maupun informal.
- Dapat menolak untuk menjawab pertanyaan di luar konteks.
- Lebih konsisten dalam menjaga batasan konteks.
- Lebih murah daripada model GPT-4.

Kelemahan : 
- Kurang konsisten dalam menjaga batasan konteks.
